### LangGraph v0.6 introduces a new durability argument that gives you fine-grained control over persistence behavior.

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver 
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini") 

In [ ]:
def call_llm(state: MessagesState):
    ai_msg = model.invoke(state["messages"])
    return {"messages": [ai_msg]}

builder = StateGraph(MessagesState)
builder.add_node("llm", call_llm)
builder.add_edge(START, "llm")
builder.add_edge("llm", END)

checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [ ]:
from langchain_core.messages import HumanMessage

config_exit = {"configurable": {"thread_id": "demo-exit"}}
out_exit = graph.invoke(
    {"messages": HumanMessage("Durability: exit")},
    config=config_exit,
    durability="exit",
)

In [ ]:
exit_history = list(graph.get_state_history(config=config_exit))
print("EXIT checkpoints:", len(exit_history))
exit_history

In [ ]:
config_async = {"configurable": {"thread_id": "demo-async"}}
out_async = graph.invoke(
    {"messages": HumanMessage("Durability: async")},
    config=config_async,
    durability="async",
)

In [ ]:
async_history = list(graph.get_state_history(config=config_async))
print("ASYNC checkpoints:", len(async_history))
async_history

In [ ]:
from langchain_core.messages import HumanMessage

config_sync = {"configurable": {"thread_id": "demo-sync"}}
out_sync = graph.invoke(
    {"messages": HumanMessage("Durability: sync")},
    config=config_sync,
    durability="sync",
)

In [ ]:
sync_history = list(graph.get_state_history(config=config_sync))
print("SYNC checkpoints:", len(sync_history))
sync_history